In [9]:
%pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import json
# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', None)  # None means unlimited
pd.set_option('display.max_columns', None)  # None means unlimited

# Your DataFrame operations here

# Reset options to default (if needed)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [11]:
from IPython.display import HTML

def titled_df(df, company_id): 
    display(HTML(f'<h2>{df.name} for {company_id}</h2>'))
    display(df)

In [12]:
#  clean up the invalid JSON files so they can be reformatted, read as JSON, or by pandas as needed.
import re

# Specify the path to your text file
file_path = '../static/ubo.json'
file_path_after =  '../static/ubo_valid.json'

# Read the content of the file
with open(file_path, 'r') as file:
    file_content = file.read()

# Define the regular expression patterns for replacements
quote_pattern = re.compile(r"'")
none_pattern = re.compile(r'\bNone\b')
true_pattern = re.compile(r'\bTrue\b')

# Perform the replacements
modified_content = quote_pattern.sub('"', file_content)
modified_content = none_pattern.sub('null', modified_content)
modified_content = true_pattern.sub('true', modified_content)


# Write the modified content back to the file
with open(file_path_after, 'w') as file:
    file.write(modified_content)

In [13]:
json_data = pd.read_json(file_path_after) # , orient="columns"
json_data.name="Paginated Output at highest level"
titled_df(json_data, "whole ubo.json")

,api_version,results
companies,0.4,[{'company': {'name': 'DAVID KENDRICK LIMITED'...
page,0.4,1
per_page,0.4,30
total_count,0.4,14
total_pages,0.4,1


In [14]:
df = pd.json_normalize(json_data['results']['companies'])
df.name = "Unmodified Dataframe"
df
titled_df(df, "whole company from ubo.json")

,company.name,company.company_number,company.jurisdiction_code,company.incorporation_date,company.dissolution_date,company.company_type,company.registry_url,company.branch,company.branch_status,company.inactive,...,company.source.retrieved_at,company.registered_address.street_address,company.registered_address.locality,company.registered_address.region,company.registered_address.postal_code,company.registered_address.country,company.registered_address_in_full,company.industry_codes,company.restricted_for_marketing,company.native_company_number
0,DAVID KENDRICK LIMITED,07178886,gb,2010-03-04,2021-03-16,Private Limited Company,https://beta.companieshouse.gov.uk/company/071...,None,None,True,...,2021-04-04T10:28:59+00:00,Nursey Bungalow\nViney Hill,Lydney,Gloucestershire,GL15 4ND,United Kingdom,"Nursey Bungalow\nViney Hill, Lydney, Glouceste...","[{'industry_code': {'code': '86.90', 'descript...",None,None


Companies has only 1 company. If `ubo.json` had multiple companies, you could iterate over each of them instead of directly querying for the main company.

In [15]:
with open(file_path_after) as f:
    data = json.load(f)
company_id = f"{data['results']['companies'][0]['company']['name']} / {data['results']['companies'][0]['company']['company_number']}"
f"Single company: {company_id}"

'Single company: DAVID KENDRICK LIMITED / 07178886'

In [16]:
# Load the JSON data
with open(file_path_after) as f:
    data = json.load(f)

# extract the only company
company = data['results']['companies'][0]['company']

# handle these separately
registered_address = company.pop('registered_address')
industry_codes = company.pop('industry_codes')
prev_names = company.pop('previous_names')

#explode could also be used.

# Clean up the registered address column
company_norm = pd.json_normalize(company)
company_norm.name="Trimmed Company Details into several tables:"
titled_df(company_norm,company_id)


,name,company_number,jurisdiction_code,incorporation_date,dissolution_date,company_type,registry_url,branch,branch_status,inactive,...,updated_at,retrieved_at,opencorporates_url,registered_address_in_full,restricted_for_marketing,native_company_number,source.publisher,source.url,source.terms,source.retrieved_at
0,Trimmed Company Details into several tables:,07178886,gb,2010-03-04,2021-03-16,Private Limited Company,https://beta.companieshouse.gov.uk/company/071...,None,None,True,...,2021-04-04T10:28:59+00:00,2021-04-04T10:28:59+00:00,https://opencorporates.com/companies/gb/07178886,"Nursey Bungalow\nViney Hill, Lydney, Glouceste...",None,None,UK Companies House,http://xmlgw.companieshouse.gov.uk/,UK Crown Copyright,2021-04-04T10:28:59+00:00


In [17]:

pref_names_df = pd.json_normalize(prev_names)
pref_names_df.name="Previous Names"
titled_df(pref_names_df,company_id)


""


In [18]:

registered_address_df = pd.json_normalize(registered_address)
registered_address_df.name="Registered Address"
titled_df(registered_address_df,company_id)


,street_address,locality,region,postal_code,country
0,Nursey Bungalow\nViney Hill,Lydney,Gloucestershire,GL15 4ND,United Kingdom


In [19]:
# Load the JSON data
with open(file_path_after) as f:
    data = json.load(f)
# Extract relevant data for industry codes of 
industry_data = data['results']['companies'][0]['company']['industry_codes']
industry_data_df = pd.json_normalize(industry_data)
industry_data_df.name = "Expanded Industry Codes column"
titled_df(industry_data_df, company_id)

,industry_code.code,industry_code.description,industry_code.code_scheme_id,industry_code.code_scheme_name,industry_code.uid
0,86.90,Other human health activities,uk_sic_2007,UK SIC Classification 2007,uk_sic_2007-8690
1,86.90,Other human health activities,eu_nace_2,European Community NACE Rev 2,eu_nace_2-8690
2,8690,Other human health activities,isic_4,UN ISIC Rev 4,isic_4-8690


In [20]:
# Load the JSON data
with open(file_path_after) as f:
    data = json.load(f)
# Extract relevant data for industry codes of 
company_source = data['results']['companies'][0]['company']['source']
company_source_df = pd.json_normalize(company_source)
company_source_df.name=f"Company Source for "
titled_df(company_source_df, company_id)


,publisher,url,terms,retrieved_at
0,UK Companies House,http://xmlgw.companieshouse.gov.uk/,UK Crown Copyright,2021-04-04T10:28:59+00:00
